In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Medical Cost Charges Prediction using Linear Regression

The Insurance dataset has been used here to build a model for predicting the charges based on the various attributes. Here I am using the **statsmodel** API to build and evaluate the model. I am using the RFE technique to identify the best features for feature selection. The model gave an adjusted R-squared value of 0.866.

In [ ]:
df = pd.read_csv('../input/insurance/insurance.csv')

In [ ]:
df.head()

In [ ]:
df.info()

#### Distribution of various features in the dataset

In [ ]:
print("Sex Value Distribution")
print(df.sex.value_counts())
print("\n")
print("Smoker Value Distribution")
print(df.smoker.value_counts())
print("\n")
print("Region Value Distribution")
print(df.region.value_counts())

## Creating dummies for the categorical columns in the dataset

In [ ]:
smoker = pd.get_dummies(df['smoker'],drop_first=True)
sex = pd.get_dummies(df['sex'],drop_first=True)
region = pd.get_dummies(df['region'],drop_first=True)
children = pd.get_dummies(df['children'],drop_first=True)
df = pd.concat([df,smoker,sex,region,children],axis=1)

In [ ]:
df.info()

### Plotting a pair plot for the numerical values to identify if any linear relationship exists between the independent and dependent variables

In [ ]:
sns.pairplot(df[['age','bmi','charges']])
plt.show()

### Box plot to see the distributions of the categorical variables

In [ ]:
plt.figure(figsize=(18, 12))
plt.subplot(2,3,1)
sns.boxplot(x = 'sex', y = 'charges', data = df)
plt.subplot(2,3,2)
sns.boxplot(x = 'children', y = 'charges', data = df)
plt.subplot(2,3,3)
sns.boxplot(x = 'smoker', y = 'charges', data = df)
plt.subplot(2,3,4)
sns.boxplot(x = 'region', y = 'charges', data = df)
plt.show()

# Data Preparation

### Dropping the categorical variables after creating dummy columns for each of those columns

In [ ]:
df = df.drop(['sex','smoker','region'],axis=1)

### Test Train Split for building/evaluating the Linear model

In [ ]:
np.random.seed(0)

df_train,df_test = train_test_split(df,train_size = 0.7,test_size = 0.3, random_state = 100)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
model_columns = df_train.columns

df_train[model_columns] = scaler.fit_transform(df_train[model_columns])
df_test[model_columns] = scaler.transform(df_test[model_columns])

In [ ]:
plt.figure(figsize = (12, 10))
sns.heatmap(df_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

In [ ]:
plt.figure(figsize=[6,6])
plt.scatter(df_train.bmi, df_train.charges)
plt.show()

In [ ]:
plt.figure(figsize=[6,6])
plt.scatter(df_train.age, df_train.charges)
plt.show()

In [ ]:
y_train = df_train.pop('charges')
X_train = df_train

In [ ]:
X_train_lm = sm.add_constant(X_train)
lr = sm.OLS(y_train, X_train_lm).fit()
lr.summary()

In [ ]:
X_train = X_train.drop('male',axis=1)
lr_2 = sm.OLS(y_train,X_train).fit()
lr_2.summary()

In [ ]:
X_train = X_train.drop([2,3,4,5],axis=1)
lr_3 = sm.OLS(y_train,X_train).fit()
lr_3.summary()

In [ ]:
test_columns_drop = ['male',2,3,4,5]

y_test = df_test.pop('charges')
x_test = df_test

x_test = x_test.drop(test_columns_drop,axis=1)
x_test_lm = sm.add_constant(x_test)

In [ ]:
lr_2.params

In [ ]:
y_pred = lr_3.predict(x_test)
#x_test_lm.columns

### Dist Plot to see the error terms be centered almost around the center

In [ ]:
sns.distplot(y_test-y_pred)